# Introduction 
In this tutorial we describe how we can tackle the famous traveling sales man problem (see [1]) with haskell. This problem is classified as a np-hard problem and therefore we will be using heuristic search algorithms to find a local optimal solution. Imagine that we are a traveling sales man and we want to visit as many cities as possible to sell as much as possible. The goal is to find a route such that we have traveled the shortest distance while visiting all the cities. 

Knowing that the problem can not be solved directly we must come up with a different approach. One of the naive approaches would be to assume a random route as the optimal route. However, we will quickly discover that trying a few random routes will results in different total distances traveled and probably only one route will be the optimal route. Ok, then we will just generate a dozen random routes and see which one has the shortest total distance and pick that one. 

With simulated annealing we are able to improve this concept, we will still be starting with a random route but selecting a new route is done in a more clever way resulting in an algorithm that's able to find a pretty short route, although it is probably not the shortest route you'll still be able to beat a lot of other traveling salesman. 

First I'll take you through some mathematical background before we start writing code.

# Chaining it together
In order to write the simulated annealing algorithm we need to understand how markov chains work and the metropolis-hastings algorithm. 

A famous markov chain is the random walk also know as the drunkard's walk[3] this markov chain simulates a drunkard walking behavior. If we consider an simplified version then the drunkard can only move in the x and y direction by one step of size $l$ at a time. The markov chain describes the transition probability of the current state $(x,y)$ to $(x',y')$ and samples a next state. Repeating this process simulates a drunkard walking. 

This is a fun example however the algorithm is brilliant, it allows us to simulate a complex random variable. The random variable could be a vector containing many dependencies between the items in the vector. The markov chain can perfectly describe the transition probabilities and after simulating the chain for a while we are able to simulate this complex random variable.

Okay, now some mathematics :). We assume that the system has a finite possible number of states and each state has a positive transition probability from the current state, i.e. **the chain is irreducable**. Important is that the transition probabilites are independent of the past state and the sum of all transition probabilities must be equal to 1. 

$$\sum Q_{ij} = 1$$
$$ Q_{ij} > 0$$

with $Q_{ij}$ as the transition probability from state $i$ to state $j$. Starting in a random initial state we are able to sample a transition and repeat this process $n$ times. The probability that the system is in state $i$ in the $n$ steps we simulated is expressed as

$$\pi(i) = \frac{1}{n}\sum_k 1 I\{k==i\}$$

with $I\{k==i\}$ as an indicator function. In the limiting case $n \rightarrow \infty$ we get for every possible state a probability that the system is in state $i$. These probabilities are called the **stationary probabilities**. 

We define $\pi(j)$ as the stationary probability that the system is in state $j$. We assume that the probability of $\pi(j)$ is defined as

$$\pi(j) = \frac{z(j)}{Z}$$

with $z(j)$ as a postive probability and $Z$ as the normalization partition. We want to define a markov chain thus we need to define the matrix $Q_{i,j}$ which describe the transition probabilities from state $i$ to state $j$. Thus we define two possible scenarios we jump from state $i$ to state $j$ given $i \neq j$ or we stay in the same state.
According to Ross [2] we can the transition probabilities as

$$Q_{i,j} = p(i,j)a(i,j) \text{ if }  i \neq j$$
$$Q_{i,i} = p(i,i) + \sum_{k\neq i} p(i,k)(1-\alpha(i,k))$$

with $p(i,j)$ as the transition probability. Using the time-reversability property we get

$$\pi(j)Q_{i,j} = \pi(j)Q_{j,i}$$

substituting the transition probabilities gives for $i \neq j$

$$\pi(j)p(i,j)a(i,j) = \pi(i)p(j,i)a(j,i)$$
$$\alpha(i,j) = \frac{\pi(i)p(j,i)}{\pi(j)p(i,j)}$$

We also know that $\pi(j) = z(j)/Z$ thus we get

$$\alpha(i,j) = min \bigg (\frac{z(j)p(j,i)}{z(i)p(i,j)},1 \bigg )$$

notice that we do not require the normalization factor $Z$ anymore which is nice because in most cases $Z$ is hard to compute or unknown.

Next up is the Metropolis-Hastings algorithm. This algorithm is a clever way of obtaining sample items from a hard to sample probability distribution. We will generate a markov chain which has as a limiting scenario the stationary probabilities from which we would like to sample. We start as follows (see Ross[2])

1. Start with a random state $X_0$ from all possible states
2. Generate a random next state candidate state $X_{c}$ according to the transition probabilities $p(X_n,j)$. 
3. Generate an uniform random between 0 and 1
4. if $U < (z(j)p(X_n, X_{c}))/(z(i)p(X_{c},X_n)$  then $X_n = X_c$ else $X_n = X_n $
5. repeat from step 3.

with the Metropolis-Hasting algorithm we are able to simulate the random variable. The first few iterations are not independent of the past state and violate one of the assumptions (stationary probability). Therefore, we should throw away the first $n$ iterations also known as the burn-in time of the algorithm. After the burn-in time the algorithm can be used as if we were sampling from a probability distribution with stationary probabilities $\pi(j)$. 

# Simulated annealing
With the simulated annealing algorithm we use the concept of the metropolis-hasting algorithm not to sample from a complex probability distribution but to explorer a possible state-space. 

In the traveling salesman problem the state space are all the possible routes we can take. Exploiting all possible states will quickly result in a combinatorial explosion. However, if we use the markov-chain metropolis-hasting algorithm we could design a clever heuristic to find an "optimal" solution. 

With the markov chain metropolis-hasting algorithm we can take samples from a hard to sample distribution. Thus, if we can come up with an acceptance probability that mimics a probability distribution with most of its density around the optimal solution we must be able to do a random walk and with a bit of luck end up in the optimal region. 

In a more formal definition we define a function V(x) and $x \in A$ with $A$ as the input space. Then we are interested in finding (see Ross[2])
$$ V^* = min_{x\in A} V(X)$$
then we can define a set $M$
$$M=\{x\in A : V(x) = V^*\}$$
Thus we want to find the elements $x$ such that $V(X)=V^*$. Remember that with the Metropolis-Hastings algorithm we are able to simulate a complex random variable. Thus we want to define a distribution such that it stimulates finding the minimum of V for elements $x$.

Let the distribution be defined as (see Boltzmann [4] for more information)
$$ P_\lambda(x) = \frac{e^{\lambda(V^*-V(x))}}{|M| + \sum_{x \not \in M} e^{\lambda(V^*-V(x))}}$$
Thus we have a distribution that if the value is close to $V^*$ the probability is high and if the distance is large then we have a large negative value in the exponent and a low probability. We created a distribution which simulated the behavior we want. However, we are not using Metropolis-Hastings to sample from this distribution but we are using it to cleverly explorer the space $A$ to find $V^*$. 

The $\lambda$ is called the cooling factor and if we take $\lambda \rightarrow \infty$ we get
$$p_\lambda(x) = \frac{\delta (x, A)}{|M|}$$
and $\delta (x,A)=1$ if $x$ is an element of $M$. Observe that if we have a value equal to $V^*$ we have a $0$ value in $e^{\lambda(V^*-V(x))}=1$ if $V(x)$ is larger than $V^*$ and $\lambda \rightarrow \infty$ we get 0.

Thus, we verified that for an increasing $\lambda$ we have a distribution that mimics the behavior of finding $x \in M$ which contains the objective $V^*$.

There is still one thing we need to tackle and that is generating candidate states (step 2 in the Metropolis-Hastings algorithm). Generating new candidate states for the traveling salesman problem is rather simple. We assume that each city has a number and we have one array that is the current route, for example, [1,2,3,4] means that we travel from city 1 to 2 and then from 2 to 3 etc. Let [1,2,3,4] be the current state then we select randomly two indices from that array, for example indices 3 and 1. Then we swap those elements and obtain [3,2,1,4]. For this new candidate state we check if the total distance traveled is shorter than the distance of route [1,2,3,4]. If this is true, then [3,2,1,4] is the new current state. If this is not true then we use the acceptance probability 

$$(1+i)^{V(x)-V(x_c)}$$

with $V(x)$ the total distance of the current route and $V(x_c)$ the total distance of the candidate route. This is slightly different from the Metropolis-Hasting algorithm were only an acceptance probability is used to accept or reject the new candidate state (Hence the cleverness of this algorithm).

Now, we have modified the Metropolis-Hastings algorithm in such a way that we can explorer all posible states $x \in A$ if the distance of the new route is shorter we accept it immediatly and otherwise with some probability (the acceptance probability). This allows us to jump out of local optima which makes this algorithm really powerfull. Another clever trick is that the algorithm cools down, meaning that after running for a while it is less likely to jump out of a local optima then it used to before. All these properties give us a powerfull heuristic function that is able to explorer the state-space cleverly and tries to find the shortest route for the traveling salesman.

In the next section the math is translated into haskell code :)

# The coding part

Reading the CSV, creating own data type called City

In [1]:
data City = City {latitude :: Maybe Float,
                 longtitude :: Maybe Float,
                 name :: Maybe String,
                 description :: Maybe String
                 } deriving Show
type Lat = Float
type Long = Float

Next we write some code to read the city.csv file and fill the data structure

In [2]:
parseCities (Left err) = []
parseCities (Right csv) = 
    foldl(\a record -> if length record == 5 then (parseCity record):a else a) [] csv
    
parseCity record = City { latitude = getFloat $ record !! 0,
                        longtitude = getFloat $ record !! 1,
                        name = getString $ record !! 2,
                        description = getString $ record !! 3}

getFloat :: String -> Maybe Float
getFloat str = case reads str::[(Float,String)] of 
                    [(val, "")] -> Just val
                    _ -> Nothing
              
                
getString :: String -> Maybe String
getString str = if null str then Nothing else Just str

We will create helper functions to create a distance matrix which converts the latitude longtitude coordinates into km distance between any two cities. 

In [3]:
deg2rad x = 2 * pi * x / 360 

distanceMatrix :: [(Lat, Long)] -> [[Float]]
distanceMatrix cities = [[distance (x,y) | x <- cities] | y <- cities ]

distance :: ((Lat, Long), (Lat, Long)) -> Float 
distance ((lat1,long1), (lat2, long2)) = radius_earth*c
                                    where 
                                        lat     = deg2rad (lat1 - lat2)
                                        long    = deg2rad (long1 - long2)
                                        a       = ((sin lat/2)**2) + cos (deg2rad lat1) * cos (deg2rad lat2) * (sin long/2)**2
                                        c       = 2 * ( atan2  (sqrt a) (sqrt (1-a)))
                                        radius_earth = 6378.7


Next part is the simulated annealing, we define a simulate function which performs one simulation step over and over again. The step is to generate a new candidate state, and then determine to accept the new state or stay in the current state. 

First we will define some types and a helper function that swaps two random elements from a list.

In [7]:
import Control.Monad.Trans.State
import System.Random
import Control.Monad

type Route = [Int]
type DistanceMatrix = [[Float]]
type AcceptState = State StdGen Route


swapElem idx1 idx2 state = firstPart ++ elem2 ++ secondPart ++ elem1 ++ lastPart
                                    where
                                        splitted = splitAt idx1 state 
                                        firstPart = case fst splitted of
                                            [] -> []
                                            xs -> init xs
                                        elem1  = case fst splitted of
                                            [] -> []
                                            xs -> [last xs] --swap element

                                        splitted2 = splitAt (idx2 - (length $ fst splitted)) $ snd splitted --split the second part
                                        secondPart = case fst splitted2 of
                                            [] -> []
                                            xs -> init xs
                                        elem2  = case fst splitted2 of
                                            [] -> []
                                            xs -> [last xs]

                                        lastPart = snd $ splitted2

swap :: Int -> Int -> [a] -> [a]
swap idx1 idx2 state | idx1 > idx2 = swapElem idx2 idx1 state 
                     | idx1 < idx2 = swapElem idx1 idx2 state
                     | otherwise = state 


Then we can define a generate new candidate function and the accept function

In [9]:
generateCandidate :: Route -> AcceptState -- stdgen route
generateCandidate route = do
                        gen <- get 
                        let (idx1, newGenerator) = randomR (1,(length route)) gen
                        let (idx2, newGenerator') = randomR (1,(length route)) newGenerator
                        put (newGenerator')
                        return $ swap idx1 idx2 route   

accept :: Int -> DistanceMatrix -> Route -> Route -> AcceptState
accept i dM route candidate = do
                        g <- get
                        let distanceRoute = totalDistance dM route 0.0
                        let distanceCandidate = totalDistance dM candidate 0.0
                        let (v, gg) =  random g :: (Float, StdGen)
                        let cn = ((1+ fromIntegral i)**(distanceRoute - distanceCandidate))
                        let route' = if distanceCandidate < distanceRoute then candidate else (if v < cn then   candidate else  route)
                        put (gg)
                        return route' 

--Helper function to calculate the total distance of the route
totalDistance :: [[Float]] -> [Int] -> Float -> Float
totalDistance dMatrix (x:y:ys) distance = totalDistance dMatrix (y:ys) distance'
                                            where
                                                distance' = distance + (dMatrix !! x !! y) --get distance from x to y
totalDistance dMatrix (x:[]) distance = distance 


Now we can simply define one simulation step, which is generating one candidate state and then decide to accept or reject the new candidate state.

In [13]:
step :: Int -> DistanceMatrix -> Route -> AcceptState 
step nr dm route = do 
    newCandidate <- generateCandidate route 
    accept nr dm route newCandidate   

And the simulation is just a loop over the simulation steps

In [12]:
sim :: DistanceMatrix -> Route -> Int -> Int -> AcceptState 
sim dm route count nCount   | count == 0 = return route
                            | otherwise = do 
                            route' <- step nCount dm route
                            sim dm route' (count - 1) (nCount+1)

Then the main function needs to load the city.csv file, parse the file in the City datastructure and then start the simulated annealing.

In [ ]:
main :: IO ()
main = do
    let filename = "cities.csv"
    input <- readFile filename
    let csv = parseCSV filename input
    let cities = parseCities csv

    let onlyLongLat = map(\city -> (latitude city, longtitude city)) cities --get lat long tuples from records
    let onlyLongLatClean = sequence $ map(\(xx,yy) -> (\x y -> (x,y)) <$> xx <*> yy) onlyLongLat 
    let dMatrix = fmap distanceMatrix onlyLongLatClean --calculate distance matrix
    let shortest = case dMatrix of
                Just dm -> evalState (sim dm (reverse [0..49]) 30000 0) (mkStdGen 10)

    let shorestDist = case dMatrix of 
            Just dM -> totalDistance dM shortest 0.0

    print shorestDist

    print shortest

For some reason the IHaskell notebook does not run this code. To run the code please download the haskell files from the GitHub repository and run it with ghci and you will see that we are able to find pretty short routes :).

GitHub repository: https://github.com/boersmamarcel/simulatedAnnealingHaskell

# References

[1] http://en.wikipedia.org/wiki/Travelling_salesman_problem

[2] http://www.amazon.com/Simulation-Fifth-Edition-Sheldon-Ross/dp/0124158250

[3] http://en.wikipedia.org/wiki/Markov_chain

[4] http://en.wikipedia.org/wiki/Boltzmann_distribution